# Jaringan Generatif

Recurrent Neural Networks (RNNs) dan varian sel ber-gate seperti Long Short Term Memory Cells (LSTMs) dan Gated Recurrent Units (GRUs) menyediakan mekanisme untuk pemodelan bahasa, yaitu mereka dapat mempelajari urutan kata dan memberikan prediksi untuk kata berikutnya dalam sebuah urutan. Hal ini memungkinkan kita menggunakan RNN untuk **tugas generatif**, seperti generasi teks biasa, terjemahan mesin, dan bahkan pembuatan keterangan gambar.

Dalam arsitektur RNN yang kita bahas di unit sebelumnya, setiap unit RNN menghasilkan keadaan tersembunyi berikutnya sebagai output. Namun, kita juga dapat menambahkan output lain ke setiap unit rekuren, yang memungkinkan kita menghasilkan sebuah **urutan** (yang panjangnya sama dengan urutan asli). Selain itu, kita dapat menggunakan unit RNN yang tidak menerima input di setiap langkah, dan hanya mengambil beberapa vektor keadaan awal, lalu menghasilkan urutan output.

Di notebook ini, kita akan fokus pada model generatif sederhana yang membantu kita menghasilkan teks. Untuk kesederhanaan, mari kita membangun **jaringan tingkat karakter**, yang menghasilkan teks huruf demi huruf. Selama pelatihan, kita perlu mengambil korpus teks, dan membaginya menjadi urutan huruf.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Membangun Kosakata Karakter

Untuk membangun jaringan generatif tingkat karakter, kita perlu memecah teks menjadi karakter individu, bukan kata. Hal ini dapat dilakukan dengan mendefinisikan tokenizer yang berbeda:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Mari kita lihat contoh bagaimana kita dapat mengenkripsi teks dari dataset kita:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Melatih RNN Generatif

Cara kita melatih RNN untuk menghasilkan teks adalah sebagai berikut. Pada setiap langkah, kita akan mengambil urutan karakter dengan panjang `nchars`, dan meminta jaringan untuk menghasilkan karakter keluaran berikutnya untuk setiap karakter masukan:

![Gambar menunjukkan contoh RNN menghasilkan kata 'HELLO'.](../../../../../translated_images/id/rnn-generate.56c54afb52f9781d.webp)

Bergantung pada skenario sebenarnya, kita mungkin juga ingin menyertakan beberapa karakter khusus, seperti *akhir urutan* `<eos>`. Dalam kasus kita, kita hanya ingin melatih jaringan untuk menghasilkan teks tanpa henti, sehingga kita akan menetapkan ukuran setiap urutan sama dengan `nchars` token. Akibatnya, setiap contoh pelatihan akan terdiri dari `nchars` masukan dan `nchars` keluaran (yang merupakan urutan masukan yang digeser satu simbol ke kiri). Minibatch akan terdiri dari beberapa urutan seperti itu.

Cara kita akan menghasilkan minibatch adalah dengan mengambil setiap teks berita dengan panjang `l`, dan menghasilkan semua kemungkinan kombinasi masukan-keluaran darinya (akan ada `l-nchars` kombinasi semacam itu). Kombinasi tersebut akan membentuk satu minibatch, dan ukuran minibatch akan berbeda pada setiap langkah pelatihan.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Sekarang mari kita definisikan jaringan generator. Jaringan ini dapat didasarkan pada sel berulang apa pun yang telah kita bahas di unit sebelumnya (simple, LSTM, atau GRU). Dalam contoh kita, kita akan menggunakan LSTM.

Karena jaringan menerima karakter sebagai input, dan ukuran kosakata cukup kecil, kita tidak memerlukan lapisan embedding, input yang dienkode satu-hot dapat langsung masuk ke sel LSTM. Namun, karena kita memberikan nomor karakter sebagai input, kita perlu mengubahnya menjadi satu-hot sebelum diteruskan ke LSTM. Hal ini dilakukan dengan memanggil fungsi `one_hot` selama proses `forward`. Encoder output akan berupa lapisan linear yang akan mengubah state tersembunyi menjadi output yang dienkode satu-hot.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Selama pelatihan, kita ingin dapat mengambil sampel teks yang dihasilkan. Untuk melakukannya, kita akan mendefinisikan fungsi `generate` yang akan menghasilkan string keluaran dengan panjang `size`, dimulai dari string awal `start`.

Cara kerjanya adalah sebagai berikut. Pertama, kita akan melewatkan seluruh string awal melalui jaringan, dan mengambil status keluaran `s` serta karakter berikutnya yang diprediksi `out`. Karena `out` dikodekan dalam bentuk one-hot, kita menggunakan `argmax` untuk mendapatkan indeks karakter `nc` dalam kosakata, lalu menggunakan `itos` untuk mengetahui karakter sebenarnya dan menambahkannya ke daftar karakter hasil `chars`. Proses menghasilkan satu karakter ini diulangi sebanyak `size` kali untuk menghasilkan jumlah karakter yang diperlukan.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Sekarang mari kita mulai pelatihan! Loop pelatihan hampir sama seperti di semua contoh sebelumnya, tetapi alih-alih mencetak akurasi, kita mencetak teks yang dihasilkan setiap 1000 epoch.

Perhatian khusus perlu diberikan pada cara kita menghitung loss. Kita perlu menghitung loss berdasarkan output yang telah dienkode satu-hot `out`, dan teks yang diharapkan `text_out`, yang merupakan daftar indeks karakter. Untungnya, fungsi `cross_entropy` mengharapkan output jaringan yang belum dinormalisasi sebagai argumen pertama, dan nomor kelas sebagai argumen kedua, yang persis seperti yang kita miliki. Fungsi ini juga secara otomatis melakukan rata-rata berdasarkan ukuran minibatch.

Kita juga membatasi pelatihan dengan sampel sebanyak `samples_to_train`, agar tidak memakan waktu terlalu lama. Kami mendorong Anda untuk bereksperimen dan mencoba pelatihan yang lebih lama, mungkin selama beberapa epoch (dalam hal ini Anda perlu membuat loop lain di sekitar kode ini).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Contoh ini sudah menghasilkan teks yang cukup baik, tetapi masih dapat ditingkatkan dalam beberapa cara:

* **Peningkatan pembuatan minibatch**. Cara kita mempersiapkan data untuk pelatihan adalah dengan menghasilkan satu minibatch dari satu sampel. Ini tidak ideal, karena ukuran minibatch berbeda-beda, dan beberapa bahkan tidak dapat dihasilkan karena teksnya lebih kecil dari `nchars`. Selain itu, minibatch yang kecil tidak cukup memanfaatkan GPU secara optimal. Akan lebih bijaksana untuk mengambil satu potongan besar teks dari semua sampel, kemudian menghasilkan semua pasangan input-output, mengacaknya, dan membuat minibatch dengan ukuran yang sama.

* **LSTM multilayer**. Ada baiknya mencoba 2 atau 3 lapisan sel LSTM. Seperti yang disebutkan di unit sebelumnya, setiap lapisan LSTM mengekstrak pola tertentu dari teks, dan dalam kasus generator tingkat karakter, kita dapat mengharapkan lapisan LSTM yang lebih rendah bertanggung jawab untuk mengekstrak suku kata, dan lapisan yang lebih tinggi - untuk kata dan kombinasi kata. Ini dapat dengan mudah diimplementasikan dengan memberikan parameter jumlah lapisan ke konstruktor LSTM.

* Anda juga mungkin ingin bereksperimen dengan **unit GRU** dan melihat mana yang memberikan performa lebih baik, serta dengan **ukuran lapisan tersembunyi yang berbeda**. Lapisan tersembunyi yang terlalu besar dapat menyebabkan overfitting (misalnya jaringan akan mempelajari teks secara persis), sedangkan ukuran yang lebih kecil mungkin tidak menghasilkan hasil yang baik.


## Generasi teks lunak dan suhu

Dalam definisi sebelumnya tentang `generate`, kita selalu memilih karakter dengan probabilitas tertinggi sebagai karakter berikutnya dalam teks yang dihasilkan. Hal ini menyebabkan teks sering "berulang" pada urutan karakter yang sama berulang kali, seperti dalam contoh berikut:
```
today of the second the company and a second the company ...
```

Namun, jika kita melihat distribusi probabilitas untuk karakter berikutnya, bisa saja perbedaan antara beberapa probabilitas tertinggi tidak terlalu besar, misalnya satu karakter memiliki probabilitas 0.2, sementara karakter lain - 0.19, dan sebagainya. Sebagai contoh, ketika mencari karakter berikutnya dalam urutan '*play*', karakter berikutnya bisa saja berupa spasi, atau **e** (seperti dalam kata *player*).

Hal ini membawa kita pada kesimpulan bahwa tidak selalu "adil" untuk memilih karakter dengan probabilitas lebih tinggi, karena memilih karakter dengan probabilitas kedua tertinggi masih bisa menghasilkan teks yang bermakna. Akan lebih bijaksana untuk **mengambil sampel** karakter dari distribusi probabilitas yang diberikan oleh keluaran jaringan.

Pengambilan sampel ini dapat dilakukan menggunakan fungsi `multinomial` yang mengimplementasikan apa yang disebut sebagai **distribusi multinomial**. Fungsi yang mengimplementasikan generasi teks **lunak** ini didefinisikan di bawah:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Kami telah memperkenalkan satu parameter lagi yang disebut **temperature**, yang digunakan untuk menunjukkan seberapa keras kita harus berpegang pada probabilitas tertinggi. Jika temperature adalah 1.0, kita melakukan sampling multinomial yang adil, dan ketika temperature meningkat tak terhingga - semua probabilitas menjadi sama, dan kita secara acak memilih karakter berikutnya. Dalam contoh di bawah ini, kita dapat mengamati bahwa teks menjadi tidak bermakna ketika kita meningkatkan temperature terlalu tinggi, dan teks tersebut menyerupai teks "berulang" yang dihasilkan secara keras ketika mendekati 0.



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan penerjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berusaha untuk memberikan hasil yang akurat, harap diingat bahwa terjemahan otomatis mungkin mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang otoritatif. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang keliru yang timbul dari penggunaan terjemahan ini.
